In [1]:
import $ivy.`org.apache.spark::spark-sql:3.3.1`
import $ivy.`sh.almond::almond-spark:0.13.2`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.sql._

import $ivy.$                                  

import $ivy.$                               


import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql._

In [2]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.Row
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions._
import org.apache.spark.sql.functions.{approx_count_distinct, col, countDistinct, percent_rank}
import org.apache.spark.sql.types.{BooleanType, IntegerType, LongType, StringType, StructField, StructType}

import org.apache.spark.sql.SparkSession

import org.apache.spark.sql.Row

import org.apache.spark.sql.expressions.Window

import org.apache.spark.sql.functions._

import org.apache.spark.sql.functions.{approx_count_distinct, col, countDistinct, percent_rank}

import org.apache.spark.sql.types.{BooleanType, IntegerType, LongType, StringType, StructField, StructType}

In [3]:
val spark = SparkSession.builder().master("local[1]")
  .appName("SparkScala")
  .getOrCreate()
import spark.implicits._
val struct = (new StructType)
  .add(StructField("id", IntegerType, false))
  .add(StructField("timestamp", IntegerType, true))
  .add(StructField("type", StringType, true))
  .add(StructField("page_id", IntegerType, true))
  .add(StructField("tag", StringType, true))
  .add(StructField("sign", BooleanType, true))
val data = Seq(
  Row(1, 1667927426, "visit", 77, "sport", true),
  Row(1, 1667327495, "click", 77, "sport", true),
  Row(1, 1667627576, "scroll", 77, "sport", true),
  Row(1, 1667427626, "move", 78, "market", true),
  Row(1, 1667135426, "visit", 77, "sport", true),
  Row(1, 1667225495, "click", 77, "sport", true),
  Row(1, 1667685576, "scroll", 77, "sport", true),
  Row(1, 1667696626, "move", 78, "market", true),
  Row(2, 1667718426, "visit", 74, "food", false),
  Row(2, 1668128495, "click", 74, "food", false),
  Row(2, 1664928576, "scroll", 74, "food", false),
  Row(2, 1667378626, "move", 77, "sport", false),
  Row(3, 1667993426, "visit", 78, "market", true),
  Row(3, 1667229495, "click", 78, "market", true),
  Row(3, 1667329576, "scroll", 78, "market", true),
  Row(3, 1667829626, "move", 77, "sport", true),
  Row(4, 1667822433, "visit", 74, "food", false),
  Row(4, 1667722455, "click", 74, "food", false),
  Row(4, 1667522526, "scroll", 74, "food", false),
  Row(4, 1667593021, "click", 78, "market", false),
  Row(4, 1667593223, "scroll", 78, "market", false),
  Row(4, 1667553727, "click", 79, "tools", false),
  Row(5, 1667443336, "visit", 78, "market", false),
  Row(6, 1667913248, "visit", 78, "market", false),
  Row(7, 1667923459, "visit", 74, "food", false),
  Row(8, 1667873542, "visit", 74, "food", false),
  Row(8, 1667623942, "visit", 74, "food", false),
  Row(9, 1667623314, "visit", 74, "food", false),
  Row(10, 1667634657, "visit", 79, "tools", false),
  Row(11, 1667643696, "visit", 77, "sport", false),
  Row(12, 1667629904, "visit", 77, "sport", false),
)
val df = spark.createDataFrame(spark.sparkContext.parallelize(data), struct)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties


22/12/01 18:49:22 INFO SparkContext: Running Spark version 3.3.1
22/12/01 18:49:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/01 18:49:22 INFO ResourceUtils: ==============================================================
22/12/01 18:49:22 INFO ResourceUtils: No custom resources configured for spark.driver.
22/12/01 18:49:22 INFO ResourceUtils: ==============================================================
22/12/01 18:49:22 INFO SparkContext: Submitted application: SparkScala
22/12/01 18:49:22 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
22/12/01 18:49:22 INFO ResourceProfile: Limiting resource is cpu
22/12/01 18:49:22 INFO ResourceProfileManager: A

spark: SparkSession = org.apache.spark.sql.SparkSession@42beb641
import spark.implicits._

struct: StructType = StructType(
  StructField("id", IntegerType, false, {}),
  StructField("timestamp", IntegerType, true, {}),
  StructField("type", StringType, true, {}),
  StructField("page_id", IntegerType, true, {}),
  StructField("tag", StringType, true, {}),
  StructField("sign", BooleanType, true, {})
)
data: Seq[Row] = List(
  [1,1667927426,visit,77,sport,true],
  [1,1667327495,click,77,sport,true],
  [1,1667627576,scroll,77,sport,true],
  [1,1667427626,move,78,market,true],
  [1,1667135426,visit,77,sport,true],
  [1,1667225495,click,77,sport,true],
  [1,1667685576,scroll,77,sport,true],
  [1,1667696626,move,78,market,true],
  [2,1667718426,visit,74,food,false],
  [2,1668128495,click,74,food,false],
  [2,1664928576,scroll,74,food,false],
  [2,1667378626,move,77,sport,false],
  [3,1667993426,visit,78,market,true],
  [3,1667229495,click,78,market,true],
  [3,1667329576,scroll,78,market,tr

In [4]:
df.printSchema()

root
 |-- id: integer (nullable = false)
 |-- timestamp: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- page_id: integer (nullable = true)
 |-- tag: string (nullable = true)
 |-- sign: boolean (nullable = true)



In [5]:
df.show()

22/12/01 18:49:41 INFO CodeGenerator: Code generated in 509.5952 ms
22/12/01 18:49:41 INFO SparkContext: Starting job: show at cmd4.sc:1
22/12/01 18:49:41 INFO DAGScheduler: Got job 0 (show at cmd4.sc:1) with 1 output partitions
22/12/01 18:49:41 INFO DAGScheduler: Final stage: ResultStage 0 (show at cmd4.sc:1)
22/12/01 18:49:41 INFO DAGScheduler: Parents of final stage: List()
22/12/01 18:49:41 INFO DAGScheduler: Missing parents: List()
22/12/01 18:49:41 INFO DAGScheduler: Submitting ResultStage 0 (MapPartitionsRDD[3] at show at cmd4.sc:1), which has no missing parents
22/12/01 18:49:42 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 24.5 KiB, free 656.4 MiB)
22/12/01 18:49:42 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 9.8 KiB, free 656.4 MiB)
22/12/01 18:49:42 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on 13de6c50d362:36135 (size: 9.8 KiB, free: 656.4 MiB)
22/12/01 18:49:42 INFO SparkContext: Crea

In [6]:
//Вывести топ-5 самых активных посетителей сайта
(df.groupBy("id").count().as("count")).orderBy(col("count").desc).show(5)

22/12/01 18:49:56 INFO CodeGenerator: Code generated in 122.1112 ms
22/12/01 18:49:57 INFO DAGScheduler: Registering RDD 5 (show at cmd5.sc:1) as input to shuffle 0
22/12/01 18:49:57 INFO DAGScheduler: Got map stage job 1 (show at cmd5.sc:1) with 1 output partitions
22/12/01 18:49:57 INFO DAGScheduler: Final stage: ShuffleMapStage 1 (show at cmd5.sc:1)
22/12/01 18:49:57 INFO DAGScheduler: Parents of final stage: List()
22/12/01 18:49:57 INFO DAGScheduler: Missing parents: List()
22/12/01 18:49:57 INFO DAGScheduler: Submitting ShuffleMapStage 1 (MapPartitionsRDD[5] at show at cmd5.sc:1), which has no missing parents
22/12/01 18:49:57 INFO MemoryStore: Block broadcast_1 stored as values in memory (estimated size 38.9 KiB, free 656.3 MiB)
22/12/01 18:49:57 INFO MemoryStore: Block broadcast_1_piece0 stored as bytes in memory (estimated size 16.6 KiB, free 656.3 MiB)
22/12/01 18:49:57 INFO BlockManagerInfo: Added broadcast_1_piece0 in memory on 13de6c50d362:36135 (size: 16.6 KiB, free: 656.

In [7]:
//Посчитать процент посетителей, у которых есть ЛК
df.groupBy("sign").count()
  .groupBy("sign")
  .agg(sum("count").alias("sum"))
  .withColumn("percentage", col("sum") * 100 / sum("sum").over()).show()

22/12/01 18:50:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/12/01 18:50:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/12/01 18:50:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/12/01 18:50:22 INFO CodeGenerator: Code generated in 60.1787 ms
22/12/01 18:50:22 INFO DAGScheduler: Registering RDD 11 (show at cmd6.sc:4) as input to shuffle 1
22/12/01 18:50:22 INFO DAGScheduler: Got map stage job 3 (show at cmd6.sc:4) with 1 output partitions
22/12/01 18:50:22 INFO DAGScheduler: Final stage: ShuffleMapStage 4 (show at cmd6.sc:4)
22/12/01 18:50:22 INFO DAGScheduler: Parents of final stage: List()
22/12/01 18:50:22 INFO DAGScheduler: Missing parents: List()
22/12/01 18:50:22 INF

In [8]:
//Вывести топ-5 страниц сайта по показателю среднего кол-ва кликов на данной странице
df.filter(df("type")==="click")
  .groupBy("page_id")
  .count()
  .orderBy( col("count").desc).show(5)

22/12/01 18:50:32 INFO CodeGenerator: Code generated in 60.0094 ms
22/12/01 18:50:32 INFO DAGScheduler: Registering RDD 20 (show at cmd7.sc:4) as input to shuffle 3
22/12/01 18:50:32 INFO DAGScheduler: Got map stage job 6 (show at cmd7.sc:4) with 1 output partitions
22/12/01 18:50:32 INFO DAGScheduler: Final stage: ShuffleMapStage 10 (show at cmd7.sc:4)
22/12/01 18:50:32 INFO DAGScheduler: Parents of final stage: List()
22/12/01 18:50:32 INFO DAGScheduler: Missing parents: List()
22/12/01 18:50:32 INFO DAGScheduler: Submitting ShuffleMapStage 10 (MapPartitionsRDD[20] at show at cmd7.sc:4), which has no missing parents
22/12/01 18:50:32 INFO MemoryStore: Block broadcast_6 stored as values in memory (estimated size 40.9 KiB, free 656.0 MiB)
22/12/01 18:50:32 INFO MemoryStore: Block broadcast_6_piece0 stored as bytes in memory (estimated size 17.4 KiB, free 656.0 MiB)
22/12/01 18:50:32 INFO BlockManagerInfo: Added broadcast_6_piece0 in memory on 13de6c50d362:36135 (size: 17.4 KiB, free: 6

In [9]:
//Добавьте столбец к фрейму данных со значением временного диапазона в рамках суток с размером окна – 4 часа(0-4, 4-8, 8-12 и т.д.)
val df2=df.withColumn("NormalDay", from_unixtime(col("timestamp"),"yyyy-MM-dd HH:mm:ss"))
  .withColumn("PartDay", (hour(from_unixtime(col("timestamp"),"yyyy-MM-dd HH:mm:ss"))/4).cast("int"))
println(df2.show())

22/12/01 18:50:40 INFO BlockManagerInfo: Removed broadcast_5_piece0 on 13de6c50d362:36135 in memory (size: 19.6 KiB, free: 656.3 MiB)
22/12/01 18:50:40 INFO BlockManagerInfo: Removed broadcast_6_piece0 on 13de6c50d362:36135 in memory (size: 17.4 KiB, free: 656.3 MiB)
22/12/01 18:50:40 INFO BlockManagerInfo: Removed broadcast_4_piece0 on 13de6c50d362:36135 in memory (size: 22.5 KiB, free: 656.3 MiB)
22/12/01 18:50:40 INFO BlockManagerInfo: Removed broadcast_2_piece0 on 13de6c50d362:36135 in memory (size: 18.0 KiB, free: 656.3 MiB)
22/12/01 18:50:40 INFO BlockManagerInfo: Removed broadcast_7_piece0 on 13de6c50d362:36135 in memory (size: 18.6 KiB, free: 656.4 MiB)
22/12/01 18:50:40 INFO BlockManagerInfo: Removed broadcast_3_piece0 on 13de6c50d362:36135 in memory (size: 16.9 KiB, free: 656.4 MiB)
22/12/01 18:50:40 INFO BlockManagerInfo: Removed broadcast_1_piece0 on 13de6c50d362:36135 in memory (size: 16.6 KiB, free: 656.4 MiB)
22/12/01 18:50:40 INFO CodeGenerator: Code generated in 34.841

df2: DataFrame = [id: int, timestamp: int ... 6 more fields]

In [10]:
//Выведите временной промежуток на основе предыдущего задания, в течение которого было больше всего активностей на сайте.
(df2.groupBy("PartDay").count().as("count"))
  .where(col("count") === (df2.groupBy("PartDay").count().as("count"))
                            .agg(max("count").alias("maxOf"))
                            .select("maxOf").collect()
                            .head.getLong(0))
  .show()

22/12/01 18:50:47 INFO CodeGenerator: Code generated in 59.1844 ms
22/12/01 18:50:47 INFO DAGScheduler: Registering RDD 28 (collect at cmd9.sc:4) as input to shuffle 4
22/12/01 18:50:47 INFO DAGScheduler: Got map stage job 9 (collect at cmd9.sc:4) with 1 output partitions
22/12/01 18:50:47 INFO DAGScheduler: Final stage: ShuffleMapStage 14 (collect at cmd9.sc:4)
22/12/01 18:50:47 INFO DAGScheduler: Parents of final stage: List()
22/12/01 18:50:47 INFO DAGScheduler: Missing parents: List()
22/12/01 18:50:47 INFO DAGScheduler: Submitting ShuffleMapStage 14 (MapPartitionsRDD[28] at collect at cmd9.sc:4), which has no missing parents
22/12/01 18:50:47 INFO MemoryStore: Block broadcast_9 stored as values in memory (estimated size 43.9 KiB, free 656.3 MiB)
22/12/01 18:50:47 INFO MemoryStore: Block broadcast_9_piece0 stored as bytes in memory (estimated size 18.9 KiB, free 656.3 MiB)
22/12/01 18:50:47 INFO BlockManagerInfo: Added broadcast_9_piece0 in memory on 13de6c50d362:36135 (size: 18.9 

In [12]:
// Создайте второй фрейм данных, который будет содержать информацию о ЛК посетителя сайта со следующим списком атрибутов
val struct2 = (new StructType)
  .add(StructField("id", IntegerType, false))
  .add(StructField("User_id", IntegerType, true))
  .add(StructField("FIO", StringType, true))
  .add(StructField("dateofcreate", IntegerType, true))

val data2 = Seq(
  Row(1, 1, "Иванов Иван Иванович", 1665023314),
  Row(2, 2, "Петров Петр Петрович", 1664123314),
  Row(3, 3, "Сергеев Сергей Сергеевич", 1665923314),
  Row(4, 4, "Сидоров Сидор Сидорович", 1661723314),
  Row(5, 5, "Дмитриев Дмитрий Дмитриевич", 1666823314),
)
val dfUs = spark.createDataFrame(spark.sparkContext.parallelize(data2), struct2)

struct2: StructType = StructType(
  StructField("id", IntegerType, false, {}),
  StructField("User_id", IntegerType, true, {}),
  StructField("FIO", StringType, true, {}),
  StructField("dateofcreate", IntegerType, true, {})
)
data2: Seq[Row] = List(
  [1,1,Иванов Иван Иванович,1665023314],
  [2,2,Петров Петр Петрович,1664123314],
  [3,3,Сергеев Сергей Сергеевич,1665923314],
  [4,4,Сидоров Сидор Сидорович,1661723314],
  [5,5,Дмитриев Дмитрий Дмитриевич,1666823314]
)
dfUs: DataFrame = [id: int, User_id: int ... 2 more fields]

In [13]:
//Вывести фамилии посетителей, которые читали хотя бы одну новость про спорт.
df.join(dfUs, df("id") === dfUs("User_id"), "inner")
  .filter(df("tag") === "sport")
  .select("FIO").distinct.show(false)

22/12/01 18:51:44 INFO CodeGenerator: Code generated in 7.362 ms
22/12/01 18:51:44 INFO DAGScheduler: Registering RDD 45 (show at cmd12.sc:3) as input to shuffle 7
22/12/01 18:51:44 INFO DAGScheduler: Got map stage job 14 (show at cmd12.sc:3) with 1 output partitions
22/12/01 18:51:44 INFO DAGScheduler: Final stage: ShuffleMapStage 23 (show at cmd12.sc:3)
22/12/01 18:51:44 INFO DAGScheduler: Parents of final stage: List()
22/12/01 18:51:44 INFO DAGScheduler: Missing parents: List()
22/12/01 18:51:44 INFO DAGScheduler: Submitting ShuffleMapStage 23 (MapPartitionsRDD[45] at show at cmd12.sc:3), which has no missing parents
22/12/01 18:51:44 INFO MemoryStore: Block broadcast_14 stored as values in memory (estimated size 25.5 KiB, free 656.1 MiB)
22/12/01 18:51:44 INFO MemoryStore: Block broadcast_14_piece0 stored as bytes in memory (estimated size 10.6 KiB, free 656.1 MiB)
22/12/01 18:51:44 INFO CodeGenerator: Code generated in 36.0994 ms
22/12/01 18:51:44 INFO BlockManagerInfo: Added bro

root
 |-- id: integer (nullable = false)
 |-- User_id: integer (nullable = true)
 |-- FIO: string (nullable = true)
 |-- dateofcreate: integer (nullable = true)

